## This is for storing the knowledge graph multi-dimension embeddings generated using PyKeen of text and image in FAISS index using Langchain

In [1]:
! pip install -q faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 48.0 MB/s eta 0:00:00


### Loading the kaggle text and image CLIP Traditional vector embeddings from Kaggle 512 dim

In [6]:
!kaggle datasets download -d anantjain1223/clip-text-coyo-1k #text
!kaggle datasets download -d anantjain1223/clip-image-coyo-1k #image

Dataset URL: https://www.kaggle.com/datasets/anantjain1223/clip-text-coyo-1k
License(s): MIT
clip-text-coyo-1k.zip: Skipping, found more recently modified local copy (use --force to force download)
Dataset URL: https://www.kaggle.com/datasets/anantjain1223/clip-image-coyo-1k
License(s): MIT
  0% 0.00/2.83M [00:00<?, ?B/s]
100% 2.83M/2.83M [00:00<00:00, 151MB/s]


In [7]:
!unzip "/content/clip-text-coyo-1k.zip"
!unzip "/content/clip-image-coyo-1k.zip"

Archive:  /content/clip-text-coyo-1k.zip
replace clip_text_embeddings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: clip_text_embeddings.csv  
replace clip_text_pca.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: clip_text_pca.csv       
replace clip_text_tsne.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: clip_text_tsne.csv      
replace clip_text_umap csv.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: clip_text_umap csv.csv  
Archive:  /content/clip-image-coyo-1k.zip
  inflating: clip_embeddings_512.csv  
  inflating: clip_image_pca.csv      
  inflating: clip_image_tsne.csv     
  inflating: clip_image_umap.csv     


## Import the .csv embeddinhgs and ocnvert them into numpy array

In [17]:
import pandas as pd
import numpy as np

text_embedding_path="/content/clip_text_embeddings.csv"
img_embedding_path="/content/clip_embeddings_512.csv"

embeddings_img=pd.read_csv(img_embedding_path)
embeddings_text=pd.read_csv(text_embedding_path)

embeddings_img.drop(columns=['url','text','Unnamed: 0'],inplace=True)
embeddings_text.drop(columns=['prompt',"Unnamed: 0"],inplace=True)

#embeddings_img=np.load("/content/image_embeddings_train1k.npy")

embeddings_img_array=np.array(embeddings_img,dtype=np.float32) # load the image embedding
embeddings_text_array=np.array(embeddings_text,dtype=np.float32) #load the text embedding

### Storing combined image and text index

In [19]:
import faiss

dimension=512
index=faiss.IndexFlatL2(dimension)

index.add(embeddings_img_array) #add the img embedding in faiss
index.add(embeddings_text_array) # add text embedding in faiss

faiss.write_index(index, 'faiss_CLIP_traditional_vector_embedding.index')

### Storing individual image and text index

In [20]:
index_img=faiss.IndexFlatL2(dimension)
index_img.add(embeddings_img_array)
faiss.write_index(index_img, 'faiss_CLIP_img_TVE.index')

index_text=faiss.IndexFlatL2(dimension)
index_text.add(embeddings_text_array)
faiss.write_index(index_text, 'faiss_CLIP_text_TVE.index')

## Loading index for similarity search query

In [ ]:
index= faiss.read_index("/content/faiss_traditional_vector_embedding.index")
index
query_embedding=np.random.rand(1,512).astype(np.float32)
query_embedding
distance,indeces= index.search(query_embedding,k=5)
distance